### Carregamento

In [4]:
import pandas as pd
# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV com separador ';'
    dados = pd.read_csv(caminho_arquivo, sep=';')
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados
# Caminho
caminho_arquivo = '../data/raw/marketing_campaign.csv'
# Carrega os dados com o separador correto
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,...,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.0,0,1,2012-11-13,34,235,...,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.0,1,0,2013-05-08,32,76,...,8,0,0,0,0,0,0,3,11,0
8,4855,1974,PhD,Together,30351.0,1,0,2013-06-06,19,14,...,9,0,0,0,0,0,0,3,11,1
9,5899,1950,PhD,Together,5648.0,1,1,2014-03-13,68,28,...,20,1,0,0,0,0,0,3,11,0


### Tamanho base

In [9]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 2240, 'Número de Colunas': 29}

### Verificação valores nulos

In [10]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)
Income,24,1.071429


### Valores nulos: Income

In [18]:
def substituir_valores_nulos(df, colunas=None, estrategia='media'):
    """
    Substitui valores nulos em colunas específicas de um DataFrame e retorna informações detalhadas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado
    - colunas (list, opcional): Lista de colunas para substituir valores nulos
    - estrategia (str): Estratégia de substituição ('media', 'mediana', 'moda')
    
    Retorna:
    - tuple: (DataFrame tratado, dict com informações das alterações)
    """
    df_tratado = df.copy()
    info_alteracoes = {}
    
    if colunas is None:
        colunas = df_tratado.select_dtypes(include=['int64', 'float64']).columns
    
    for coluna in colunas:
        if coluna in df_tratado.columns:
            nulos_antes = df_tratado[coluna].isnull().sum()
            
            if nulos_antes > 0:
                if estrategia == 'media':
                    valor = df_tratado[coluna].mean()
                elif estrategia == 'mediana':
                    valor = df_tratado[coluna].median()
                elif estrategia == 'moda':
                    valor = df_tratado[coluna].mode()[0]
                else:
                    raise ValueError("Estratégia inválida. Use 'media', 'mediana' ou 'moda'")
                
                # Correção do warning: usar loc para atribuição
                df_tratado.loc[df_tratado[coluna].isnull(), coluna] = valor
                
                info_alteracoes[coluna] = {
                    'valores_alterados': nulos_antes,
                    'valor_substituicao': round(valor, 2),
                    'estrategia': estrategia
                }
    
    # Exibir resumo das alterações
    for coluna, info in info_alteracoes.items():
        print(f"\nAlteração realizada na coluna: {coluna}")
        print(f"Valores nulos substituídos: {info['valores_alterados']}")
        print(f"Valor de substituição ({info['estrategia']}): {info['valor_substituicao']}")
    
    if not info_alteracoes:
        print("\nNenhuma alteração foi necessária nas colunas especificadas.")
    
    return df_tratado, info_alteracoes

# Aplicar substituição de valores nulos
dados, info_alteracoes = substituir_valores_nulos(dados, colunas=['Income'])

# Verificar se ainda existem valores nulos
verificar_valores_ausentes(dados)


Nenhuma alteração foi necessária nas colunas especificadas.


,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [13]:
def verificar_valores_duplicados(df, colunas=None):
    """
    Verifica valores duplicados em um DataFrame, fornecendo análise detalhada.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado
    - colunas (list, opcional): Lista específica de colunas para verificar duplicatas.
                               Se None, verifica todas as colunas.
    
    Retorna:
    - dict: Dicionário com informações sobre duplicatas para cada coluna analisada
    """
    resultado = {}
    
    # Se nenhuma coluna específica for fornecida, usar todas as colunas
    if colunas is None:
        colunas = df.columns
    
    # Análise de duplicatas por coluna
    for coluna in colunas:
        # Encontrar valores duplicados
        duplicados = df[df[coluna].duplicated(keep='first')]
        qtd_duplicados = len(duplicados)
        
        if qtd_duplicados > 0:
            # Contagem de ocorrências de cada valor duplicado
            valores_duplicados = df[coluna].value_counts()[df[coluna].value_counts() > 1]
            
            resultado[coluna] = {
                'possui_duplicados': True,
                'quantidade_duplicados': qtd_duplicados,
                'valores_e_contagem': valores_duplicados.to_dict()
            }
        else:
            resultado[coluna] = {
                'possui_duplicados': False,
                'quantidade_duplicados': 0,
                'valores_e_contagem': {}
            }
    
    # Exibir resumo
    print("=== Análise de Valores Duplicados ===\n")
    
    for coluna, info in resultado.items():
        print(f"\nColuna: {coluna}")
        if info['possui_duplicados']:
            print(f"Status: Possui {info['quantidade_duplicados']} valores duplicados")
            print("Valores duplicados e suas ocorrências:")
            for valor, contagem in info['valores_e_contagem'].items():
                print(f"  - Valor: {valor} | Ocorrências: {contagem}")
        else:
            print("Status: Não possui valores duplicados")
    
    return resultado

# Executar verificação de valores duplicados
resultados_duplicados = verificar_valores_duplicados(dados)

=== Análise de Valores Duplicados ===


Coluna: ID
Status: Não possui valores duplicados

Coluna: Year_Birth
Status: Possui 2181 valores duplicados
Valores duplicados e suas ocorrências:
  - Valor: 1976 | Ocorrências: 89
  - Valor: 1971 | Ocorrências: 87
  - Valor: 1975 | Ocorrências: 83
  - Valor: 1972 | Ocorrências: 79
  - Valor: 1978 | Ocorrências: 77
  - Valor: 1970 | Ocorrências: 77
  - Valor: 1973 | Ocorrências: 74
  - Valor: 1965 | Ocorrências: 74
  - Valor: 1969 | Ocorrências: 71
  - Valor: 1974 | Ocorrências: 69
  - Valor: 1956 | Ocorrências: 55
  - Valor: 1958 | Ocorrências: 53
  - Valor: 1979 | Ocorrências: 53
  - Valor: 1952 | Ocorrências: 52
  - Valor: 1977 | Ocorrências: 52
  - Valor: 1968 | Ocorrências: 51
  - Valor: 1959 | Ocorrências: 51
  - Valor: 1966 | Ocorrências: 50
  - Valor: 1954 | Ocorrências: 50
  - Valor: 1955 | Ocorrências: 49
  - Valor: 1960 | Ocorrências: 49
  - Valor: 1982 | Ocorrências: 45
  - Valor: 1963 | Ocorrências: 45
  - Valor: 1967 | Ocorrências: 

### Verificação tipo dos dados

In [15]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,ID,int64
1,Year_Birth,int64
2,Education,object
3,Marital_Status,object
4,Income,float64
5,Kidhome,int64
6,Teenhome,int64
7,Dt_Customer,object
8,Recency,int64
9,MntWines,int64


### Verificação dos dados

In [16]:
def verificar_colunas_numericas(df, colunas):
    """
    Verifica se as colunas especificadas contêm apenas valores numéricos.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    - colunas (list): Lista de nomes das colunas a serem verificadas.

    Retorna:
    - dict: Dicionário com o nome da coluna e um booleano indicando se contém apenas números.
    - pandas.DataFrame: DataFrame com as inconsistências encontradas.
    """
    resultado = {}
    inconsistencias = []

    for coluna in colunas:
        if coluna in df.columns:
            # Verifica se todos os valores na coluna são numéricos
            col_numerica = pd.to_numeric(df[coluna], errors='coerce')
            resultado[coluna] = col_numerica.notnull().all()
            if not resultado[coluna]:
                # Adiciona as inconsistências à lista
                posicoes_inconsistentes = col_numerica[col_numerica.isnull()].index.tolist()
                for posicao in posicoes_inconsistentes:
                    inconsistencias.append({'Coluna': coluna, 'Posição': posicao, 'Valor': df.at[posicao, coluna]})
        else:
            resultado[coluna] = False
            print(f"⚠️ Coluna '{coluna}' não encontrada no DataFrame.")

    # Cria um DataFrame com as inconsistências
    df_inconsistencias = pd.DataFrame(inconsistencias)

    return resultado, df_inconsistencias

# Colunas a serem verificadas
colunas_a_verificar = [
    'ID', 'Year_Birth', 'Income', 
    'Kidhome', 'Teenhome', 'Recency',
    'MntWines', 'MntFruits', 'MntMeatProducts',
    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 
    'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 
    'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 
    'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 
    'AcceptedCmp2', 'Complain', 'Z_CostContact', 
    'Z_Revenue', 'Response'

]

# Verificar as colunas no DataFrame 'dados'
resultado_verificacao, df_inconsistencias = verificar_colunas_numericas(dados, colunas_a_verificar)

# Exibir o resultado da verificação
for coluna, is_numerica in resultado_verificacao.items():
    status = "contém apenas números" if is_numerica else "não contém apenas números"
    print(f"Coluna '{coluna}': {status}")

# Exibir as inconsistências, se houver
if not df_inconsistencias.empty:
    print("\nInconsistências encontradas:")
    display(df_inconsistencias)
else:
    print("\nNenhuma inconsistência encontrada nas colunas verificadas.")

Coluna 'ID': contém apenas números
Coluna 'Year_Birth': contém apenas números
Coluna 'Income': contém apenas números
Coluna 'Kidhome': contém apenas números
Coluna 'Teenhome': contém apenas números
Coluna 'Recency': contém apenas números
Coluna 'MntWines': contém apenas números
Coluna 'MntFruits': contém apenas números
Coluna 'MntMeatProducts': contém apenas números
Coluna 'MntFishProducts': contém apenas números
Coluna 'MntSweetProducts': contém apenas números
Coluna 'MntGoldProds': contém apenas números
Coluna 'NumDealsPurchases': contém apenas números
Coluna 'NumWebPurchases': contém apenas números
Coluna 'NumCatalogPurchases': contém apenas números
Coluna 'NumStorePurchases': contém apenas números
Coluna 'NumWebVisitsMonth': contém apenas números
Coluna 'AcceptedCmp3': contém apenas números
Coluna 'AcceptedCmp4': contém apenas números
Coluna 'AcceptedCmp5': contém apenas números
Coluna 'AcceptedCmp1': contém apenas números
Coluna 'AcceptedCmp2': contém apenas números
Coluna 'Compla

### Valores máx/min

In [17]:
def valores_max_min(df):
    """
    Percorre todas as colunas numéricas (int ou float) de um DataFrame e retorna uma tabela com os valores máximos e mínimos de cada coluna.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores máximos e mínimos de cada coluna numérica.
    """
    # Selecionar apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Criar listas para armazenar os resultados
    colunas = []
    valores_min = []
    valores_max = []
    
    # Percorrer cada coluna numérica e calcular os valores máximo e mínimo
    for coluna in colunas_numericas:
        colunas.append(coluna)
        valores_min.append(df[coluna].min())
        valores_max.append(df[coluna].max())
    
    # Criar o DataFrame final com todos os resultados de uma vez
    tabela_max_min = pd.DataFrame({
        'Coluna': colunas,
        'Valor Mínimo': valores_min,
        'Valor Máximo': valores_max
    })
    
    return tabela_max_min

# Aplicar a função e exibir a tabela de valores máximos e mínimos
tabela_max_min = valores_max_min(dados)
tabela_max_min

,Coluna,Valor Mínimo,Valor Máximo
0,ID,0.0,11191.0
1,Year_Birth,1893.0,1996.0
2,Income,1730.0,666666.0
3,Kidhome,0.0,2.0
4,Teenhome,0.0,2.0
5,Recency,0.0,99.0
6,MntWines,0.0,1493.0
7,MntFruits,0.0,199.0
8,MntMeatProducts,0.0,1725.0
9,MntFishProducts,0.0,259.0


### Conversão/otimização

In [19]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'ID': 'int16',  # de 0 a 11191
    'Year_Birth': 'int16',  # de 1893 a 1996
    'Income': 'float32',  # de 1730 a 666666
    'Kidhome': 'int8',  # de 0 a 2
    'Teenhome': 'int8',  # de 0 a 2
    'Recency': 'int8',  # de 0 a 99
    'MntWines': 'int16',  # de 0 a 1493
    'MntFruits': 'int16',  # de 0 a 199
    'MntMeatProducts': 'int16',  # de 0 a 1725
    'MntFishProducts': 'int16',  # de 0 a 259
    'MntSweetProducts': 'int16',  # de 0 a 263
    'MntGoldProds': 'int16',  # de 0 a 362
    'NumDealsPurchases': 'int8',  # de 0 a 15
    'NumWebPurchases': 'int8',  # de 0 a 27
    'NumCatalogPurchases': 'int8',  # de 0 a 28
    'NumStorePurchases': 'int8',  # de 0 a 13
    'NumWebVisitsMonth': 'int8',  # de 0 a 20
    'AcceptedCmp3': 'int8',  # de 0 a 1
    'AcceptedCmp4': 'int8',  # de 0 a 1
    'AcceptedCmp5': 'int8',  # de 0 a 1
    'AcceptedCmp1': 'int8',  # de 0 a 1
    'AcceptedCmp2': 'int8',  # de 0 a 1
    'Complain': 'int8',  # de 0 a 1
    'Z_CostContact': 'int8',  # valor constante 3
    'Z_Revenue': 'int8',  # valor constante 11
    'Response': 'int8'  # de 0 a 1
}
 

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'ID' convertida para 'int16' com sucesso.
✅ Coluna 'Year_Birth' convertida para 'int16' com sucesso.
✅ Coluna 'Income' convertida para 'float32' com sucesso.
✅ Coluna 'Kidhome' convertida para 'int8' com sucesso.
✅ Coluna 'Teenhome' convertida para 'int8' com sucesso.
✅ Coluna 'Recency' convertida para 'int8' com sucesso.
✅ Coluna 'MntWines' convertida para 'int16' com sucesso.
✅ Coluna 'MntFruits' convertida para 'int16' com sucesso.
✅ Coluna 'MntMeatProducts' convertida para 'int16' com sucesso.
✅ Coluna 'MntFishProducts' convertida para 'int16' com sucesso.
✅ Coluna 'MntSweetProducts' convertida para 'int16' com sucesso.
✅ Coluna 'MntGoldProds' convertida para 'int16' com sucesso.
✅ Coluna 'NumDealsPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumWebPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumCatalogPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumStorePurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumWebVisitsMonth' convertid

ID                       int16
Year_Birth               int16
Education               object
Marital_Status          object
Income                 float32
Kidhome                   int8
Teenhome                  int8
Dt_Customer             object
Recency                   int8
MntWines                 int16
MntFruits                int16
MntMeatProducts          int16
MntFishProducts          int16
MntSweetProducts         int16
MntGoldProds             int16
NumDealsPurchases         int8
NumWebPurchases           int8
NumCatalogPurchases       int8
NumStorePurchases         int8
NumWebVisitsMonth         int8
AcceptedCmp3              int8
AcceptedCmp4              int8
AcceptedCmp5              int8
AcceptedCmp1              int8
AcceptedCmp2              int8
Complain                  int8
Z_CostContact             int8
Z_Revenue                 int8
Response                  int8
dtype: object

### Remoção espaços

In [20]:
def remover_espacos(df):
    """
    Remove espaços em branco à esquerda e à direita das strings em todas as colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os espaços removidos.
    - pandas.DataFrame: DataFrame com informações sobre as remoções realizadas.
    """
    # Verificar se o DataFrame é válido
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame.")
    
    # Inicializar lista para armazenar informações sobre remoções
    informacoes_remocoes = []

    # Aplicar strip em todas as colunas de tipo object e category
    for col in df.select_dtypes(include=['object', 'category']).columns:
        # Contar espaços em branco antes da remoção
        espacos_antes = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Remover espaços em branco
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
        
        # Contar espaços em branco após a remoção
        espacos_depois = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Verificar se a remoção foi bem-sucedida
        sucesso = espacos_depois == 0
        
        # Adicionar informações à lista
        informacoes_remocoes.append({
            'Coluna': col,
            'Espaços Antes': espacos_antes,
            'Espaços Depois': espacos_depois,
            'Remoção Bem-Sucedida': sucesso
        })
    
    # Criar DataFrame com as informações de remoção
    df_informacoes_remocoes = pd.DataFrame(informacoes_remocoes)
    
    return df, df_informacoes_remocoes

# Exemplo de uso:
dados, informacoes_remocoes = remover_espacos(dados)

# Exibir as informações sobre as remoções
print("Informações sobre a remoção de espaços em branco:")
display(informacoes_remocoes)

Informações sobre a remoção de espaços em branco:


,Coluna,Espaços Antes,Espaços Depois,Remoção Bem-Sucedida
0,Education,0,0,True
1,Marital_Status,0,0,True
2,Dt_Customer,0,0,True


### Conversão coluna Dt_Customer

In [24]:
# Convert Dt_Customer to datetime
try:
    dados['Dt_Customer'] = pd.to_datetime(dados['Dt_Customer'])
    print("✅ Conversão da coluna 'Dt_Customer' para datetime realizada com sucesso!")
    
    # Show updated data types
    print("\nVisão geral dos tipos de dados após a conversão:")
    display(dados.dtypes)
except Exception as e:
    print(f"❌ Erro na conversão: {e}")


✅ Conversão da coluna 'Dt_Customer' para datetime realizada com sucesso!

Visão geral dos tipos de dados após a conversão:


ID                              int16
Year_Birth                      int16
Education                      object
Marital_Status                 object
Income                        float32
Kidhome                          int8
Teenhome                         int8
Dt_Customer            datetime64[ns]
Recency                          int8
MntWines                        int16
MntFruits                       int16
MntMeatProducts                 int16
MntFishProducts                 int16
MntSweetProducts                int16
MntGoldProds                    int16
NumDealsPurchases                int8
NumWebPurchases                  int8
NumCatalogPurchases              int8
NumStorePurchases                int8
NumWebVisitsMonth                int8
AcceptedCmp3                     int8
AcceptedCmp4                     int8
AcceptedCmp5                     int8
AcceptedCmp1                     int8
AcceptedCmp2                     int8
Complain                         int8
Z_CostContac

### Valores únicos

In [25]:
def detectar_valores_unicos(df, coluna):
    """
    Detecta valores únicos em uma coluna de um DataFrame e exibe esses valores de forma amigável.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.
    - coluna (str): Nome da coluna cujos valores únicos serão detectados.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores únicos e suas contagens.
    - str: Mensagem indicando o sucesso da operação e o número de valores únicos encontrados.
    """
    if coluna not in df.columns:
        raise ValueError(f"Coluna '{coluna}' não encontrada no DataFrame.")
    
    # Detectar valores únicos e suas contagens
    valores_unicos = df[coluna].value_counts().reset_index()
    valores_unicos.columns = [coluna, 'Contagem']
    
    # Criar a mensagem de saída
    mensagem = f"Operação bem-sucedida! Foram encontrados {valores_unicos.shape[0]} valores únicos na coluna '{coluna}'."
    
    return valores_unicos, mensagem

# Exemplo de uso:
colunas = ['Education', 'Marital_Status', 'Dt_Customer']
for coluna in colunas:
    valores_unicos, mensagem = detectar_valores_unicos(dados, coluna)
    # Exibir a mensagem e a tabela de valores únicos
    print(mensagem)
    display(valores_unicos)

Operação bem-sucedida! Foram encontrados 5 valores únicos na coluna 'Education'.


,Education,Contagem
0,Graduation,1127
1,PhD,486
2,Master,370
3,2n Cycle,203
4,Basic,54


Operação bem-sucedida! Foram encontrados 8 valores únicos na coluna 'Marital_Status'.


,Marital_Status,Contagem
0,Married,864
1,Together,580
2,Single,480
3,Divorced,232
4,Widow,77
5,Alone,3
6,Absurd,2
7,YOLO,2


Operação bem-sucedida! Foram encontrados 663 valores únicos na coluna 'Dt_Customer'.


,Dt_Customer,Contagem
0,2012-08-31,12
1,2012-09-12,11
2,2013-02-14,11
3,2014-05-12,11
4,2013-08-20,10
...,...,...
658,2012-08-05,1
659,2012-11-18,1
660,2013-05-25,1
661,2013-04-14,1


### Conversão/otimização

In [26]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'Marital_Status':'category',
    'Education': 'category'
}
 

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados

✅ Coluna 'Marital_Status' convertida para 'category' com sucesso.
✅ Coluna 'Education' convertida para 'category' com sucesso.


ID                              int16
Year_Birth                      int16
Education                    category
Marital_Status               category
Income                        float32
Kidhome                          int8
Teenhome                         int8
Dt_Customer            datetime64[ns]
Recency                          int8
MntWines                        int16
MntFruits                       int16
MntMeatProducts                 int16
MntFishProducts                 int16
MntSweetProducts                int16
MntGoldProds                    int16
NumDealsPurchases                int8
NumWebPurchases                  int8
NumCatalogPurchases              int8
NumStorePurchases                int8
NumWebVisitsMonth                int8
AcceptedCmp3                     int8
AcceptedCmp4                     int8
AcceptedCmp5                     int8
AcceptedCmp1                     int8
AcceptedCmp2                     int8
Complain                         int8
Z_CostContac

### Carga

In [28]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/marketing_campaign_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/marketing_campaign_atualizado.csv'
